# Setting Up the Database from the comments dump file

In [5]:
import sqlite3
import json
import pandas as pd
from datetime import datetime

In [1]:


timeframe = '2017-11'

sql_transaction = []

connection =sqlite3.connect('{}.db'.format(timeframe))
c= connection.cursor()

In [2]:
def create_table():
  c.execute("""CREATE TABLE IF NOT EXISTS parent_reply
  (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT,
  comment TEXT, subreddit TEXT, unix INT, score INT)""")

def format_data(data):
  data = data.replace("\n"," newlinechar ").replace("\r", " newlinechar ").replace('"', "'")
  return data

def find_parent(parent_id):
    try:
    
        sql = " SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(parent_id)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
              return result[0]
        else: return False
    except Exception as e:
        return False

def find_existing_score(pid):
    try:
    
        sql = " SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(parent_id)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
              return result[0]
        else: return False
    except Exception as e:
        return False

def acceptable(data):
    if len(data.split(" ")) > 50 or len(data) < 1:   #might wanna change that to a different value
        return False
    elif len(data)>1000:
        return False
    elif data == "[deleted]" or data == "[removed]":
        return False
    else:
        return True

def sql_insert_replace_comment(commentid, parentid, parent,comment,subreddit, time , score):
    try:
        sql = """ UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?
        , unix = ?, score = ? WHERE parent_id =?; """.format(parentid, commentid, parent, comment, subreddit,int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print(' replace comment ', str(e))
        
def sql_insert_has_parent(commentid, parentid, parent,comment,subreddit, time , score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES 
        ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print(' s-PARENT ', str(e))
        
def sql_insert_no_parent(commentid, parentid,comment,subreddit, time, score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES
        ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print(' s-NOPARENT ', str(e))
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [4]:
if __name__ == "__main__":
    create_table()
    row_counter = 0
    paired_rows = 0
  
    with open("RC_2017-11", buffering = 1000) as f:
       
        for row in f:
            try:
                row_counter += 1
                row = json.loads(row)
                parent_id = row['parent_id']
                try:
                    comment_id = row['name']
                except KeyError:
                    comment_id = 't1_' + row['id']
                body = format_data(row['body'])
                created_utc = row['created_utc']
                score = row['score']
                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)

                if row_counter % 100000 == 0:
                        print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))



                if score >= 2 and score !=  None:
                    if acceptable(body):
                        existing_comment_score = find_existing_score(parent_id)
                        if existing_comment_score:
                            if score> existing_comment_score:
                                sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit,created_utc, score)

                        else:
                            if parent_data:
                                sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit,created_utc, score)
                                paired_rows+=1
                            else:
                                sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
            except Exception as e:
                pass
#             cleanup = 10000000
#             if row_counter > start_row:
#                 if row_counter % cleanup == 0:
#                     print("Cleaning up!")
#                     print("Turning journal off temporarily...")
#                     c.execute("PRAGMA journal_mode = OFF")
#                     print("Deleting null parent rows...")
#                     sql = "DELETE FROM parent_reply WHERE parent IS NULL"
#                     c.execute(sql)
#                     print("committing changes")
#                     connection.commit()
#                     print("vacuuming...")
#                     c.execute("VACUUM")
#                     print("committing...")
#                     connection.commit()
#                     print("Turning journal back on (\"MEMORY\" mode)...")
#                     c.execute("PRAGMA journal_mode = MEMORY")
#                     connection.commit()
#                     print("Back to loading and processing new rows!\n")

Total Rows Read: 100000, Paired Rows: 0, Time: 2019-01-12 17:06:46.278320
Total Rows Read: 200000, Paired Rows: 0, Time: 2019-01-12 17:07:22.362782
Total Rows Read: 300000, Paired Rows: 0, Time: 2019-01-12 17:07:57.502587
Total Rows Read: 400000, Paired Rows: 0, Time: 2019-01-12 17:08:28.755666
Total Rows Read: 500000, Paired Rows: 0, Time: 2019-01-12 17:08:57.133389
Total Rows Read: 600000, Paired Rows: 0, Time: 2019-01-12 17:09:52.640856
Total Rows Read: 700000, Paired Rows: 0, Time: 2019-01-12 17:10:39.022328
Total Rows Read: 800000, Paired Rows: 0, Time: 2019-01-12 17:11:27.122302
Total Rows Read: 900000, Paired Rows: 0, Time: 2019-01-12 17:12:04.753559
Total Rows Read: 1000000, Paired Rows: 0, Time: 2019-01-12 17:12:42.398051
Total Rows Read: 1100000, Paired Rows: 0, Time: 2019-01-12 17:13:19.897151
Total Rows Read: 1200000, Paired Rows: 0, Time: 2019-01-12 17:14:01.873467
Total Rows Read: 1300000, Paired Rows: 0, Time: 2019-01-12 17:14:47.410572
Total Rows Read: 1400000, Paired R

Total Rows Read: 11100000, Paired Rows: 0, Time: 2019-01-12 21:56:41.463984
Total Rows Read: 11200000, Paired Rows: 0, Time: 2019-01-12 21:57:17.640513
Total Rows Read: 11300000, Paired Rows: 0, Time: 2019-01-12 21:57:53.841601
Total Rows Read: 11400000, Paired Rows: 0, Time: 2019-01-12 21:58:28.083874
Total Rows Read: 11500000, Paired Rows: 0, Time: 2019-01-12 21:59:06.592635
Total Rows Read: 11600000, Paired Rows: 0, Time: 2019-01-12 21:59:41.150203
Total Rows Read: 11700000, Paired Rows: 0, Time: 2019-01-12 22:00:21.741399
Total Rows Read: 11800000, Paired Rows: 0, Time: 2019-01-12 22:00:56.378788
Total Rows Read: 11900000, Paired Rows: 0, Time: 2019-01-12 22:01:30.203440
Total Rows Read: 12000000, Paired Rows: 0, Time: 2019-01-12 22:02:04.555874
Total Rows Read: 12100000, Paired Rows: 0, Time: 2019-01-12 22:02:40.859166
Total Rows Read: 12200000, Paired Rows: 0, Time: 2019-01-12 22:03:19.709695
Total Rows Read: 12300000, Paired Rows: 0, Time: 2019-01-12 22:03:58.398880
Total Rows R

Total Rows Read: 21900000, Paired Rows: 55849, Time: 2019-01-12 23:20:47.495724
Total Rows Read: 22000000, Paired Rows: 62972, Time: 2019-01-12 23:22:32.054137
Total Rows Read: 22100000, Paired Rows: 70106, Time: 2019-01-12 23:23:47.879551
Total Rows Read: 22200000, Paired Rows: 77495, Time: 2019-01-12 23:25:31.028925
Total Rows Read: 22300000, Paired Rows: 84838, Time: 2019-01-12 23:26:54.566802
Total Rows Read: 22400000, Paired Rows: 92410, Time: 2019-01-12 23:28:18.478836
Total Rows Read: 22500000, Paired Rows: 99947, Time: 2019-01-12 23:29:43.781620
Total Rows Read: 22600000, Paired Rows: 107502, Time: 2019-01-12 23:31:32.304675
Total Rows Read: 22700000, Paired Rows: 114684, Time: 2019-01-12 23:32:57.341460
Total Rows Read: 22800000, Paired Rows: 121898, Time: 2019-01-12 23:34:29.878533
Total Rows Read: 22900000, Paired Rows: 128988, Time: 2019-01-12 23:35:51.862924
Total Rows Read: 23000000, Paired Rows: 135985, Time: 2019-01-12 23:37:39.786507
Total Rows Read: 23100000, Paired R

Total Rows Read: 32100000, Paired Rows: 755997, Time: 2019-01-13 01:41:28.546088
Total Rows Read: 32200000, Paired Rows: 762083, Time: 2019-01-13 01:42:40.240835
Total Rows Read: 32300000, Paired Rows: 768682, Time: 2019-01-13 01:43:52.109733
Total Rows Read: 32400000, Paired Rows: 775378, Time: 2019-01-13 01:45:02.349615
Total Rows Read: 32500000, Paired Rows: 782224, Time: 2019-01-13 01:46:13.385874
Total Rows Read: 32600000, Paired Rows: 789176, Time: 2019-01-13 01:47:22.266381
Total Rows Read: 32700000, Paired Rows: 796285, Time: 2019-01-13 01:48:43.392914
Total Rows Read: 32800000, Paired Rows: 803693, Time: 2019-01-13 01:49:55.591027
Total Rows Read: 32900000, Paired Rows: 810216, Time: 2019-01-13 01:50:59.810409
Total Rows Read: 33000000, Paired Rows: 816518, Time: 2019-01-13 01:52:03.092149
Total Rows Read: 33100000, Paired Rows: 823181, Time: 2019-01-13 01:53:07.248982
Total Rows Read: 33200000, Paired Rows: 829324, Time: 2019-01-13 01:54:14.141955
Total Rows Read: 33300000, P

Total Rows Read: 42200000, Paired Rows: 1448079, Time: 2019-01-13 04:06:42.512117
Total Rows Read: 42300000, Paired Rows: 1455316, Time: 2019-01-13 04:08:37.767127
Total Rows Read: 42400000, Paired Rows: 1462434, Time: 2019-01-13 04:10:29.610301
Total Rows Read: 42500000, Paired Rows: 1469739, Time: 2019-01-13 04:12:25.500176
Total Rows Read: 42600000, Paired Rows: 1476931, Time: 2019-01-13 04:14:23.243891
Total Rows Read: 42700000, Paired Rows: 1484059, Time: 2019-01-13 04:16:23.299964
Total Rows Read: 42800000, Paired Rows: 1491327, Time: 2019-01-13 04:18:25.485311
Total Rows Read: 42900000, Paired Rows: 1498499, Time: 2019-01-13 11:00:57.963318
Total Rows Read: 43000000, Paired Rows: 1505604, Time: 2019-01-13 11:03:23.335875
Total Rows Read: 43100000, Paired Rows: 1512641, Time: 2019-01-13 11:05:40.810559
Total Rows Read: 43200000, Paired Rows: 1519297, Time: 2019-01-13 11:08:17.204779
Total Rows Read: 43300000, Paired Rows: 1526213, Time: 2019-01-13 11:10:47.455336
Total Rows Read:

Total Rows Read: 52200000, Paired Rows: 2137240, Time: 2019-01-13 14:06:20.936609
Total Rows Read: 52300000, Paired Rows: 2143468, Time: 2019-01-13 14:08:45.102606
Total Rows Read: 52400000, Paired Rows: 2149505, Time: 2019-01-13 14:10:39.980246
Total Rows Read: 52500000, Paired Rows: 2155880, Time: 2019-01-13 14:12:25.977936
Total Rows Read: 52600000, Paired Rows: 2162522, Time: 2019-01-13 14:14:40.560556
Total Rows Read: 52700000, Paired Rows: 2169294, Time: 2019-01-13 14:16:23.728125
Total Rows Read: 52800000, Paired Rows: 2176524, Time: 2019-01-13 14:17:48.262941
Total Rows Read: 52900000, Paired Rows: 2183503, Time: 2019-01-13 14:19:09.484174
Total Rows Read: 53000000, Paired Rows: 2190456, Time: 2019-01-13 14:20:34.531172
Total Rows Read: 53100000, Paired Rows: 2196989, Time: 2019-01-13 14:21:54.447856
Total Rows Read: 53200000, Paired Rows: 2203240, Time: 2019-01-13 14:23:12.650826
Total Rows Read: 53300000, Paired Rows: 2209869, Time: 2019-01-13 14:24:32.446170
Total Rows Read:

Total Rows Read: 62200000, Paired Rows: 2820499, Time: 2019-01-13 17:04:45.500004
Total Rows Read: 62300000, Paired Rows: 2827833, Time: 2019-01-13 17:06:25.611633
Total Rows Read: 62400000, Paired Rows: 2834952, Time: 2019-01-13 17:08:08.340371
Total Rows Read: 62500000, Paired Rows: 2842251, Time: 2019-01-13 17:09:44.189068
Total Rows Read: 62600000, Paired Rows: 2849533, Time: 2019-01-13 17:11:24.794498
Total Rows Read: 62700000, Paired Rows: 2856627, Time: 2019-01-13 17:12:59.278328
Total Rows Read: 62800000, Paired Rows: 2863504, Time: 2019-01-13 17:14:35.410532
Total Rows Read: 62900000, Paired Rows: 2870241, Time: 2019-01-13 17:16:10.142569
Total Rows Read: 63000000, Paired Rows: 2876799, Time: 2019-01-13 17:17:46.195499
Total Rows Read: 63100000, Paired Rows: 2883142, Time: 2019-01-13 17:19:41.886543
Total Rows Read: 63200000, Paired Rows: 2889679, Time: 2019-01-13 17:21:36.727442
Total Rows Read: 63300000, Paired Rows: 2896527, Time: 2019-01-13 17:23:33.234393
Total Rows Read:

Total Rows Read: 72200000, Paired Rows: 3500851, Time: 2019-01-13 20:05:14.226088
Total Rows Read: 72300000, Paired Rows: 3507314, Time: 2019-01-13 20:07:14.183198
Total Rows Read: 72400000, Paired Rows: 3513663, Time: 2019-01-13 20:09:11.040551
Total Rows Read: 72500000, Paired Rows: 3520408, Time: 2019-01-13 20:11:05.052316
Total Rows Read: 72600000, Paired Rows: 3527325, Time: 2019-01-13 20:13:03.336352
Total Rows Read: 72700000, Paired Rows: 3534243, Time: 2019-01-13 20:15:01.167022
Total Rows Read: 72800000, Paired Rows: 3541032, Time: 2019-01-13 20:16:57.066290
Total Rows Read: 72900000, Paired Rows: 3547631, Time: 2019-01-13 20:18:55.126212
Total Rows Read: 73000000, Paired Rows: 3554764, Time: 2019-01-13 20:21:05.504520
Total Rows Read: 73100000, Paired Rows: 3561806, Time: 2019-01-13 20:22:49.885626
Total Rows Read: 73200000, Paired Rows: 3568729, Time: 2019-01-13 20:24:33.367627
Total Rows Read: 73300000, Paired Rows: 3575444, Time: 2019-01-13 20:26:18.556873
Total Rows Read:

Total Rows Read: 82200000, Paired Rows: 4202770, Time: 2019-01-13 23:09:28.214413
Total Rows Read: 82300000, Paired Rows: 4209652, Time: 2019-01-13 23:11:35.840402
Total Rows Read: 82400000, Paired Rows: 4216519, Time: 2019-01-13 23:13:42.635384
Total Rows Read: 82500000, Paired Rows: 4223494, Time: 2019-01-13 23:15:50.666474
Total Rows Read: 82600000, Paired Rows: 4230577, Time: 2019-01-13 23:17:59.537952
Total Rows Read: 82700000, Paired Rows: 4237492, Time: 2019-01-13 23:19:59.367660
Total Rows Read: 82800000, Paired Rows: 4244289, Time: 2019-01-13 23:21:59.636921
Total Rows Read: 82900000, Paired Rows: 4250926, Time: 2019-01-13 23:24:06.605333
Total Rows Read: 83000000, Paired Rows: 4257246, Time: 2019-01-13 23:26:12.033922
Total Rows Read: 83100000, Paired Rows: 4263655, Time: 2019-01-13 23:28:10.025558
Total Rows Read: 83200000, Paired Rows: 4270413, Time: 2019-01-13 23:30:06.582888
Total Rows Read: 83300000, Paired Rows: 4277332, Time: 2019-01-13 23:32:04.222168
Total Rows Read:

# After the database has been setup


 Now time for some preprocessing of Question and Answer pair

In [7]:
timeframes = ['2017-11']

for t in timeframes:
    connection = sqlite3.connect('{}.db'.format(t))
    c = connection.cursor()
    limit = 50000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False
    while cur_length == limit:
        df = pd.read_sql("""SELECT * FROM parent_reply WHERE unix > {} 
        AND parent NOT NULL AND score > 0 ORDER BY unix ASC LIMIT {}""".format(last_unix, limit), connection)
        last_unix = df.tail(1)['unix'].values[0]
        cur_length = len(df)
        if not test_done:
            with open("test.from", 'a', encoding = 'utf8') as f:
                for content in df['parent'].values:
                    f.write(content + '\n')
            with open("test.to", 'a', encoding = 'utf8') as f:
                for content in df['comment'].values:
                    f.write(content + '\n')
            test_done = True
        else:
            with open("train.from", 'a', encoding = 'utf8') as f:
                for content in df['parent'].values:
                    f.write(content + '\n')
            with open("train.to", 'a', encoding = 'utf8') as f:
                for content in df['comment'].values:
                    f.write(content + '\n')
            test_done = True
        counter +=1
        if counter% 20 == 0:
            print(counter*limit, 'rows completed so far')
            

1000000 rows completed so far
2000000 rows completed so far
3000000 rows completed so far
4000000 rows completed so far
5000000 rows completed so far
